# Installations

In [ ]:
!pip install emoji texthero flair gdown 
!pip install textattack==0.3.0

  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
!pip install tensorflow_text

  Using cached importlib_metadata-4.8.2-py3-none-any.whl (17 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 4.8.2 which is incompatible.


In [ ]:
import pandas as pd
import re 
import emoji
import texthero as hero
from texthero import preprocessing
import zipfile
import os
import random
import gdown
from textattack.attack_recipes import DeepWordBugGao2018, Pruthi2019, TextBuggerLi2018, PSOZang2020, PWWSRen2019, TextFoolerJin2019, IGAWang2019, BAEGarg2019, CheckList2020
from textattack.loggers.csv_logger import CSVLogger
from textattack.datasets import Dataset
from flair.models import TextClassifier
import time
from flair.data import Sentence
from pandas import DataFrame
from textattack.models.wrappers import ModelWrapper



# A custom model wrapper 
* This Class is instantiated with the path to a trained model 
* An instance of this class can be called with a list of texts and it returns a list of predictions 

In [ ]:
class CustomModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model
    
    def __call__(self, text_input_list):
        df = DataFrame(text_input_list,columns=['claimReview_claimReviewed'])
        #df['clean_tweet'] = pipeline(df)
        sentences = [Sentence(sentence) for sentence in df['claimReview_claimReviewed'].to_list()]
        self.model.predict(sentences, verbose = False, mini_batch_size=32)
        predictions=[]
        for sentence in sentences:
            score = sentence.labels[0].score
            if sentence.labels[0].value == 'TRUE':
                predictions.append([1-score,score])
            else:
                predictions.append([score, 1-score])
        return predictions

# Create a custom_dataset of false claims to be used to attack the trained model 

In [ ]:
custom_dataset = []

# download data
!wget https://github.com/untruenews/ss2021/raw/main/data/data.csv.zip
zf = zipfile.ZipFile("data.csv.zip") 

# open
df_raw = pd.read_csv(zf.open('data_out.csv')).sample(n=3000, random_state=1)
df_raw["rating_alternateName_normalized"]=df_raw["rating_alternateName_normalized"].str.upper() #only making sure the labels are normalized.

# taking only the english text
train_df=df_raw[df_raw.language=="en"][["claimReview_claimReviewed","rating_alternateName_normalized"]]

for index, row in train_df[train_df['rating_alternateName_normalized']=='FALSE'].iterrows():
    custom_dataset.append((row['claimReview_claimReviewed'], 0))

--2021-11-30 23:54:54--  https://github.com/untruenews/ss2021/raw/main/data/data.csv.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/untruenews/ss2021/main/data/data.csv.zip [following]
--2021-11-30 23:54:54--  https://raw.githubusercontent.com/untruenews/ss2021/main/data/data.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85485669 (82M) [application/zip]
Saving to: ‘data.csv.zip.3’

data.csv.zip.3      100%[===================>]  81.53M   132MB/s    in 0.6s    

2021-11-30 23:54:54 (132 MB/s) - ‘data.csv.zip.3’ saved [85485669/85485669]



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
train_df

,claimReview_claimReviewed,rating_alternateName_normalized
18966,Blog says North Korea told its people their te...,FALSE
3243,Sanders says U.S. doubles every other country ...,FALSE
6637,Austin district's claim to highest graduation ...,OTHER
27880,Florida House candidate Lizbeth Benacquisto ac...,TRUE
21297,Malia Obama was expelled from Harvard for a dr...,FALSE
...,...,...
10198,Obama promised to cut the deficit in half by t...,TRUE
32485,Google Earth helped locate a woman who had bee...,FALSE
13562,Rauner distorts Dems' message on Illinois inco...,MIXTURE
93,"Burger King has ""bowed to Sharia law"" and will...",MIXTURE


In [ ]:
random.seed(44)
custom_dataset= random.sample(custom_dataset,500)

# Save your results to google drive

Please add link to trello so the others can access your results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir "/content/drive/MyDrive/checklist"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Pretrained Models & Attack

In [ ]:
import torch
recipes= ['CheckList2020']
# ['DeepWordBugGao2018','Pruthi2019','TextBuggerLi2018','PSOZang2020', 'PWWSRen2019','TextFoolerJin2019','IGAWang2019','BAEGarg2019','CheckList2020']
models = ['flair','berttweet']


for model_name in models:
  for recipe in recipes:
    print(recipe)
    print(model_name)

    recept_name = recipe
    timestr = time.strftime("%Y%m%d-%H%M%S")
    result_dir = "/content/drive/MyDrive/checklist/results-" + recept_name +"-" + timestr
    os.mkdir(result_dir)

    if model_name == 'flair':
      url = 'https://drive.google.com/uc?id=1xh7O-Wl4Rwr-eau1OrBQRCRXvZ6Z31J4' 
    elif model_name == 'berttweet':
      url = 'https://drive.google.com/uc?id=1m1Zqif7HH4alCoEPBKwXk-9gXXuP0mDh' 
    else:
      url = 'https://drive.google.com/uc?id=1-ouVsPHfIM7pscfguU_upcjZao55A02S' 
    output = 'best-model.pt' 
    gdown.download(url, output, quiet = True)

    model = TextClassifier.load('./best-model.pt') 
    model_wrapper = CustomModelWrapper(model)

    if recipe == 'DeepWordBugGao2018':  
      attack =  DeepWordBugGao2018.build(model_wrapper);
    elif recipe == 'Pruthi2019':  
      attack =  Pruthi2019.build(model_wrapper);
    elif recipe == 'TextBuggerLi2018':  
      attack =  TextBuggerLi2018.build(model_wrapper);
    elif recipe == 'PSOZang2020':  
      attack =  PSOZang2020.build(model_wrapper);
    elif recipe == 'PWWSRen2019':  
      attack =  PWWSRen2019.build(model_wrapper);
    elif recipe == 'TextFoolerJin2019':  
      attack =  TextFoolerJin2019.build(model_wrapper);
    elif recipe == 'IGAWang2019':  
      attack =  IGAWang2019.build(model_wrapper);
    elif recipe == 'BAEGarg2019':  
      attack =  BAEGarg2019.build(model_wrapper);
    elif recipe == 'CheckList2020':  
      attack =  CheckList2020.build(model_wrapper);
    
    dataset = Dataset(custom_dataset)

    # Add timestamp to result file
    logger = CSVLogger(color_method='html', filename=result_dir+'/results-'+model_name+'.csv')

    for example, label in custom_dataset:
      result = attack.attack(example, label)
      logger.log_attack_result(result)
      print(result.__str__(color_method='ansi'))

    # Write the result csv to google drive
    logger.flush()
    summary = logger.df['result_type'].value_counts()
    summary.to_csv(result_dir+'/summary-'+model_name+'.csv')

CheckList2020
flair
2021-11-30 23:55:17,583 loading file ./best-model.pt


textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/checklist/results-CheckList2020-20211130-235515/results-flair.csv


0 (72%) --> [FAILED]

Wisconsin state Sen. Scott Fitzgerald says some people have been found to be voting five or six times a day
0 (90%) --> [FAILED]

Rand Paul said that Marco Rubio is for an 'open border'
1 (53%) --> [SKIPPED]

Microsoft virus warning
0 (93%) --> [FAILED]

A U.S. surgeon who exposed “Clinton Foundation corruption in Haiti” was found dead in his home under suspicious circumstances.
0 (68%) --> [FAILED]

Michele Bachmann said that solar power drains the sun of heat.
0 (88%) --> [FAILED]

Tonopah Solar Company $737 Million Government Loan For Solar 
Project
0 (85%) --> [FAILED]

The zodiac is about to change to include a "new" star sign, Ophiuchus.
0 (55%) --> 1 (56%)

Bill Clinton’s 1996 Immigration Law Behind Child Separations

Kayleb Clinton’s 1996 Immigration Law Behind Child Separations
1 (64%) --> [SKIPPED]

Warner says Gillespie supports a personhood amendment
0 (65%) --> [FAILED]

Is Price guilty of "Tomfoolery"?
1 (54%) --> [SKIPPED]

Arizona School Requires F

textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/checklist/results-CheckList2020-20211201-010205/results-berttweet.csv


0 (96%) --> 1 (72%)

Wisconsin state Sen. Scott Fitzgerald says some people have been found to be voting five or six times a day

Wisconsin state Sen. Scott Gu says some people have been found to be voting nine or four times a day
0 (100%) --> [FAILED]

Rand Paul said that Marco Rubio is for an 'open border'
1 (100%) --> [SKIPPED]

Microsoft virus warning
0 (100%) --> [FAILED]

A U.S. surgeon who exposed “Clinton Foundation corruption in Haiti” was found dead in his home under suspicious circumstances.
0 (100%) --> [FAILED]

Michele Bachmann said that solar power drains the sun of heat.
0 (100%) --> [FAILED]

Tonopah Solar Company $737 Million Government Loan For Solar 
Project
0 (100%) --> [FAILED]

The zodiac is about to change to include a "new" star sign, Ophiuchus.
0 (100%) --> [FAILED]

Bill Clinton’s 1996 Immigration Law Behind Child Separations
0 (100%) --> [FAILED]

Warner says Gillespie supports a personhood amendment
0 (100%) --> [FAILED]

Is Price guilty of "Tomfoolery"?
0 

# Attack results 
* `logger.df` is a dataframe that has all the results of the attack
* The results can be either `Successful`, `Failed` or `Skipped`
* An attack is skipped when the original text is missclassified by the model and therefore the text won't be used to attack the model

In [ ]:
logger.df['result_type'].value_counts()

Failed        381
Successful     60
Skipped        59
Name: result_type, dtype: int64

In [ ]:
summary = logger.df['result_type'].value_counts()
summary.to_csv(result_dir+'/summary-'+model_name+'.csv')


In [ ]:
berttweet_summary = pd.read_csv("/content/drive/MyDrive/checklist/results-CheckList2020-20211201-010205/summary-berttweet.csv")
berttweet_summary

,Unnamed: 0,result_type
0,Failed,381
1,Successful,60
2,Skipped,59


In [ ]:
flair_summary = pd.read_csv("/content/drive/MyDrive/checklist/results-CheckList2020-20211130-235515/summary-flair.csv")
flair_summary

,Unnamed: 0,result_type
0,Failed,326
1,Skipped,141
2,Successful,33
